In [5]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `llama2access` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llama

In [1]:
!git clone https://github.com/vivek3141/ghostbuster.git

Cloning into 'ghostbuster'...
remote: Enumerating objects: 487, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 487 (delta 174), reused 146 (delta 146), pack-reused 284 (from 1)
Receiving objects: 100% (487/487), 6.97 MiB | 4.74 MiB/s, done.
Resolving deltas: 100% (286/286), done.


In [2]:
!git clone https://github.com/vivek3141/ghostbuster-data.git
!find . -type d -name logprobs -exec rm -r {} +
!mv ghostbuster-data/ ghostbuster/data/

Cloning into 'ghostbuster-data'...
remote: Enumerating objects: 165086, done.
remote: Counting objects: 100% (6455/6455), done.
remote: Compressing objects: 100% (6354/6354), done.
remote: Total 165086 (delta 104), reused 6398 (delta 100), pack-reused 158631 (from 1)
Receiving objects: 100% (165086/165086), 636.57 MiB | 30.59 MiB/s, done.
Resolving deltas: 100% (18707/18707), done.
Updating files: 100% (156011/156011), done.


In [ ]:
# !pip install vllm
# !python -m vllm.entrypoints.openai.api_server --model deepseek-ai/deepseek-llm-7b-base --dtype=half
# !pip install openai==0.28.1

In [3]:
import openai
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
import torch
import torch.nn.functional as F
import tqdm
import gc

import shutil
from google.colab import files

# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"

# openai.api_key = "EMPTY"
# openai.api_base = "http://0.0.0.0:8000/v1"


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "meta-llama/Llama-2-7b-hf" # "TinyLlama/TinyLlama_v1.1"
llama_tokenizer = AutoTokenizer.from_pretrained(model_name)
llama_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
    )



tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [7]:
def write_llama_logprobs(text, out, model):
  with torch.no_grad():
    encodings = llama_tokenizer(text, return_tensors="pt", max_length=2047, truncation=True).to(device)
    input_ids = encodings["input_ids"].to(device)
    output = model(input_ids)
    log_probs = F.log_softmax(output.logits, dim=-1)

    tokens = encodings["input_ids"]
    indices = torch.tensor([[[i] for i in tokens[0]]])[:, 1:, :].to(device)

    # 1 to N tokens
    target_ids = input_ids[0, 1:]
    logprob_pro_token = log_probs[0, :-1, :].gather(1, target_ids.unsqueeze(1)).squeeze(1).cpu().numpy()
    tokens = llama_tokenizer.convert_ids_to_tokens(target_ids.cpu())

  to_write = ""
  for _, (w, p) in enumerate(zip(tokens, logprob_pro_token)):
    to_write += f"{w} {-p}\n"

  with open(out, "w") as f:
    f.write(to_write)


In [7]:

def write_ds_logprobs(text, out_ds):

  tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-llm-7b-base")

  tokens = tokenizer.encode(text)

  # Check if the first token is BOS
  if tokens[0] == tokenizer.bos_token_id:
      tokens = tokens[1:]
  tokens = tokens[:2047]
  doc = tokenizer.decode(tokens)

  response = openai.Completion.create(
    model="deepseek-ai/deepseek-llm-7b-base",
      prompt=doc,
        max_tokens=0,
          echo=True,
            logprobs=1,
            )

  subwords = response["choices"][0]["logprobs"]["tokens"][1:]
  subprobs = response["choices"][0]["logprobs"]["token_logprobs"][1:]

  gpt2_map = {"\n": "Ċ", "\t": "ĉ", " ": "Ġ"}

  for i in range(len(subwords)):
    for k, v in gpt2_map.items():
      subwords[i] = subwords[i].replace(k, v)

  to_write = ""
  for _, (w, p) in enumerate(zip(subwords, subprobs)):
    to_write += f"{w} {-p}\n"

  with open(out_ds, "w") as f:
    f.write(to_write)

In [20]:
# Generate logprobs using llama models
def is_txt_file(file):
  return file.endswith(".txt")

def process_folder(folder, llama_model):
  if folder.endswith("prompts"):
    return
  # print(f"in Folder {folder}")
  # Check if folder logprob exists
  logprobs_dir = os.path.join(folder, "logprobs")
  if not os.path.exists(logprobs_dir):
    os.makedirs(logprobs_dir)
  # read all txt files
  for fname in os.listdir(folder):
    if is_txt_file(fname):
      full_path = os.path.join(folder, fname)
      print(f"Processing {full_path}")
      with open(full_path, "r") as f:
        text = f.read().strip()

      # output path
      out_llama = os.path.join(logprobs_dir, fname.replace(".txt", "-llama.txt"))
      # out_tinyllama = os.path.join(logprobs_dir, fname.replace(".txt", "-tinyllama.txt"))
      # out_ds = os.path.join(logprobs_dir, fname.replace(".txt", "-ds.txt"))

      write_llama_logprobs(text, out_llama, llama_model)
      # write_ds_logprobs(text, out_ds)

def traverse_and_generate(data_root, llama_model):
  for root, dirs, files in os.walk(data_root):

    # Skip folders which do not need generate logprobs
    if os.path.basename(root) in ("prompt", "logprobs", "headlines", "logprobs_babbage","prompts"):
      continue

    # Check if txt files contained
    if any(is_txt_file(f) for f in files):
      process_folder(root, llama_model)

# Generate Logprobs
traverse_and_generate("ghostbuster/data/essay", llama_model)

shutil.make_archive('/content/essay', 'zip', '/content/ghostbuster/data/essay')


files.download('/content/essay.zip')


流式输出内容被截断，只能显示最后 5000 行内容。
Processing ghostbuster/data/essay/gpt_writing/69.txt
Processing ghostbuster/data/essay/gpt_writing/485.txt
Processing ghostbuster/data/essay/gpt_writing/191.txt
Processing ghostbuster/data/essay/gpt_writing/837.txt
Processing ghostbuster/data/essay/gpt_writing/148.txt
Processing ghostbuster/data/essay/gpt_writing/903.txt
Processing ghostbuster/data/essay/gpt_writing/620.txt
Processing ghostbuster/data/essay/gpt_writing/178.txt
Processing ghostbuster/data/essay/gpt_writing/643.txt
Processing ghostbuster/data/essay/gpt_writing/793.txt
Processing ghostbuster/data/essay/gpt_writing/622.txt
Processing ghostbuster/data/essay/gpt_writing/784.txt
Processing ghostbuster/data/essay/gpt_writing/931.txt
Processing ghostbuster/data/essay/gpt_writing/501.txt
Processing ghostbuster/data/essay/gpt_writing/340.txt
Processing ghostbuster/data/essay/gpt_writing/54.txt
Processing ghostbuster/data/essay/gpt_writing/581.txt
Processing ghostbuster/data/essay/gpt_writing/589.txt
Pro

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.make_archive('/content/essay', 'zip', '/content/ghostbuster/data/essay')


files.download('/content/essay.zip')

In [ ]:
# Generate logprobs using deepseek models

# def is_txt_file(file):
#   return file.endswith(".txt")

# def process_folder(folder):
#   if folder.endswith("prompts"):
#     return
#   # print(f"in Folder {folder}")
#   # check if logprob folder exists
#   logprobs_dir = os.path.join(folder, "logprobs")
#   if not os.path.exists(logprobs_dir):
#       os.makedirs(logprobs_dir)
#   # read all txt files
#   for fname in os.listdir(folder):
#     if is_txt_file(fname):
#       full_path = os.path.join(folder, fname)
#       print(f"Processing {full_path}")
#       with open(full_path, "r") as f:
#         text = f.read().strip()

#       out_ds = os.path.join(logprobs_dir, fname.replace(".txt", "-ds.txt"))
#

#       write_ds_logprobs(text, out_ds)

# def traverse_and_generate(data_root):
#   for root, dirs, files in os.walk(data_root):

#
#     if os.path.basename(root) in ("prompt", "logprobs", "headlines"):
#       continue

#
#     if any(is_txt_file(f) for f in files):
#       process_folder(root)

# traverse_and_generate("ghostbuster/data/wp/human")


# shutil.make_archive('/content/human', 'zip', '/content/ghostbuster/data/wp/human')


# files.download('/content/human.zip')

In [ ]:
!for f in /content/ghostbuster/data/perturb/sent_adj/0/logprobs/*-ds.txt; do rm -v "/content/ghostbuster/data/perturb/sent_adj/0/$(basename "$f" -ds.txt).txt"; done


In [ ]:
!nvidia-smi


Sat May 24 06:57:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             51W /  400W |   26129MiB /  40960MiB |     33%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
file = "/content/1.txt"
with open(file, "r") as f:
    text = f.read().strip()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch, torch.nn.functional as F

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

inputs = tokenizer(text, return_tensors="pt").to(device)
logits = model(**inputs).logits

probs = F.softmax(logits[:, :-1], dim=-1)
target_ids = inputs.input_ids[:, 1:]
logprobs = torch.log(probs.gather(2, target_ids.unsqueeze(-1)).squeeze(-1))

